<a href="https://colab.research.google.com/github/MarVinikoi/DL_NafldReserch/blob/main/ImageClassif_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tcn
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 26.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd "/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage/"

/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage


In [ ]:
from tensorflow.python.keras import callbacks
from tcn import TCN, tcn_full_summary
from tensorflow.python.keras.models import Sequential
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import glob, os
from sklearn.model_selection import train_test_split
import math
import random
from google.colab.patches import cv2_imshow
import pandas as pd

print(tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.10/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [ ]:
%%python3 --version

cv2.__version__

'4.6.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Variáveis Globais

path_doentes = "images/doentes"
path_saudaveis = "images/nao_doentes"

In [ ]:
def main():
  #dataAugmentation(True) #Só executa com o parametro setado em True
  #paddingImages()


  #numImgs(True)
  #model = tf.keras.models.load_model("/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage/images/Melhores Modelos/model.26-0.78")
  loadData()

if __name__ == "__main__":
  main()

images/doentes/doente_1
image 0 = images/doentes/doente_1/IR_41519.jpg
image 4 = images/doentes/doente_1/IR_41527.jpg
image 7 = images/doentes/doente_1/IR_41533.jpg
images/doentes/doente_2
image 0 = images/doentes/doente_2/IR_41355.jpg
image 2 = images/doentes/doente_2/IR_41359.jpg
image 5 = images/doentes/doente_2/IR_41367.jpg
images/doentes/doente_3
image 0 = images/doentes/doente_3/IR_41023.jpg
image 2 = images/doentes/doente_3/IR_41027.jpg
image 5 = images/doentes/doente_3/IR_41033.jpg
images/doentes/doente_4
image 0 = images/doentes/doente_4/IR_43059.jpg
image 2 = images/doentes/doente_4/IR_43063.jpg
image 5 = images/doentes/doente_4/IR_43069.jpg
images/doentes/doente_5
image 0 = images/doentes/doente_5/IR_41473.jpg
image 6 = images/doentes/doente_5/IR_41485.jpg
image 11 = images/doentes/doente_5/IR_41497.jpg
images/doentes/doente_6
image 0 = images/doentes/doente_6/IR_41455.jpg
image 4 = images/doentes/doente_6/IR_41465.jpg
image 7 = images/doentes/doente_6/IR_41471.jpg
images/do

Epoch 2/50
3/3 [==============================] - 1s 381ms/step - loss: 5080.6851 - accuracy: 0.4688 - val_loss: 123.2020 - val_accuracy: 0.5000
Epoch 3/50
3/3 [==============================] - 1s 391ms/step - loss: 1512.7915 - accuracy: 0.4896 - val_loss: 2623.3093 - val_accuracy: 0.5000
Epoch 4/50
3/3 [==============================] - 1s 381ms/step - loss: 2224.8030 - accuracy: 0.5521 - val_loss: 1353.8590 - val_accuracy: 0.5000
Epoch 5/50
3/3 [==============================] - 1s 386ms/step - loss: 1192.9430 - accuracy: 0.4271 - val_loss: 513.2789 - val_accuracy: 0.5000
Epoch 6/50
3/3 [==============================] - 1s 364ms/step - loss: 354.9997 - accuracy: 0.5625 - val_loss: 1579.4159 - val_accuracy: 0.5000
Epoch 7/50
3/3 [==============================] - 1s 378ms/step - loss: 956.1525 - accuracy: 0.5521 - val_loss: 1395.1018 - val_accuracy: 0.5000
Epoch 8/50
3/3 [==============================] - 1s 388ms/step - loss: 996.9187 - accuracy: 0.4479 - val_loss: 1856.4756 - val_

Epoch 22/50
3/3 [==============================] - 1s 395ms/step - loss: 128.7562 - accuracy: 0.7708 - val_loss: 188.6119 - val_accuracy: 0.5000
Epoch 23/50
3/3 [==============================] - 2s 588ms/step - loss: 53.1174 - accuracy: 0.8229 - val_loss: 161.5566 - val_accuracy: 0.3750
Epoch 24/50
3/3 [==============================] - 2s 630ms/step - loss: 28.4335 - accuracy: 0.8854 - val_loss: 184.9464 - val_accuracy: 0.6250
Epoch 25/50
3/3 [==============================] - 1s 466ms/step - loss: 11.8872 - accuracy: 0.9583 - val_loss: 518.4237 - val_accuracy: 0.5000
Epoch 26/50
3/3 [==============================] - 1s 445ms/step - loss: 7.9820 - accuracy: 0.9167 - val_loss: 227.1754 - val_accuracy: 0.6250
Epoch 27/50
3/3 [==============================] - 1s 384ms/step - loss: 12.3178 - accuracy: 0.9479 - val_loss: 248.4702 - val_accuracy: 0.6250
Epoch 28/50
3/3 [==============================] - 1s 396ms/step - loss: 3.5990 - accuracy: 0.9792 - val_loss: 288.1565 - val_accuracy: 

TypeError: ignored

In [ ]:

#Função encarregada de carregar os dados em memória e chamar a função para realização do treino.
def loadData(model = None):
  timesteps = 3
  treino = []
  teste = []
  val = []

  labels_treino = []
  labels_teste = []
  labels_val = []


  vetor_imgs_doe = []
  vetor_imgs_sau = []
  vetor_final_imgs = []

  #Carregando os vetores de imagens e os transformando em séries temporais
  #Vetor de Doentes
  for paciente in os.listdir(path_doentes):
    path_paciente = os.path.join(path_doentes, paciente)
    print(path_paciente)
    paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
    for i, img_path in enumerate(paths_imgs):
      #Se a imagem for a primeira, última ou do meio, iremos operar sobre ela
      if(img_path == paths_imgs[0] or img_path == paths_imgs[-1] or img_path == paths_imgs[round((len(paths_imgs)-1)/2)]):
        image = cv2.imread(img_path)
        print(f"image {i} = {img_path}")
        
        vetor_imgs_doe.append(image)

  #LABELS --> 0 = DOENTES  ||  1 = SAUDÁVEIS
  #labels.extend(np.zeros(len(vetor_final_imgs)/timesteps), dtype = "int32")

  #Vetor de Saudáveis
  for paciente in os.listdir(path_saudaveis):
    path_paciente = os.path.join(path_saudaveis, paciente)
    paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
    print(path_paciente)
    for i, img_path in enumerate(paths_imgs):
      #Se a imagem for a primeira, última ou do meio, iremos operar sobre ela
      if(img_path == paths_imgs[0] or img_path == paths_imgs[-1] or img_path == paths_imgs[round((len(paths_imgs)-1)/2)]):
        
        image = cv2.imread(img_path)
        vetor_imgs_sau.append(image)
        print(f"image {i} = {img_path}")


  #LABELS --> 0 = DOENTES  ||  1 = SAUDÁVEIS
  #labels.extend(np.ones(len(vetor_final_imgs) - len(labels)))



  print(f"Len vetor final de doentes: {len(vetor_imgs_doe)}")
  print(f"Len vetor final de saudáveis: {len(vetor_imgs_sau)}")

  
  treino_doe = []
  treino_sau = []
  #Utilizando cerca de 60% das imagens para treino
  treino_doe.extend(vetor_imgs_doe[:round(len(vetor_imgs_doe)*0.6)])
  treino_sau.extend(vetor_imgs_sau[:round(len(vetor_imgs_sau)*0.6)])

  treino, labels_treino = dataAugmentation(treino_doe, treino_sau)

  #Utilizando cerca de 20% das imagens para teste
  teste.extend(vetor_imgs_doe[round(len(vetor_imgs_doe)*0.6):round(len(vetor_imgs_doe)*0.8)])
  labels_teste = [0 for x in range(len(teste))]
  teste.extend(vetor_imgs_sau[round(len(vetor_imgs_sau)*0.6):round(len(vetor_imgs_sau)*0.8)])
  labels_teste.extend(1 for x in range(len(teste)-len(labels_teste)))


  #Utilizando cerca de 15% das imagens para validação
  val.extend(vetor_imgs_doe[round(len(vetor_imgs_doe)*0.8):])
  labels_val = [0 for x in range(len(val))]
  val.extend(vetor_imgs_sau[round(len(vetor_imgs_sau)*0.8):])
  labels_val.extend(1 for x in range(len(val)-len(labels_val)))


  #Conversão para Séries Temporais

  
  print("Estado dos conjuntos:")
  print(f"Treino: {len(treino)} -- Labels: {len(labels_treino)}")
  print(f"Teste: {len(teste)} -- Labels: {len(labels_teste)}")
  print(f"Val: {len(val)} -- Labels: {len(labels_val)}")

  treino, teste, val, labels_treino, labels_teste, labels_val = toTemporal(treino, teste, val, labels_treino, labels_teste, labels_val)


  treino = np.array(treino)
  teste = np.array(teste)
  val = np.array(val)

  print(treino.shape)
  print(teste.shape)
  print(val.shape)



  #Embaralhando os vetores de imagens térmicas
  
  random.seed(852741)

  zipped_treino = list(zip(treino, labels_treino))
  zipped_teste = list(zip(teste, labels_teste))
  zipped_val = list(zip(val, labels_val))


  random.shuffle(zipped_treino)
  random.shuffle(zipped_teste)
  random.shuffle(zipped_val)

  treino, labels_treino = zip(*zipped_treino)
  teste, labels_teste = zip(*zipped_teste)
  val, labels_val = zip(*zipped_val)



  print(f"Labels de Treino: {labels_treino}")
  print(f"Labels de Teste: {labels_teste}")
  print(f"Labels de Validação: {labels_val}")
  
  treino = np.array(treino)
  teste = np.array(teste)
  val = np.array(val)
  labels_treino = np.array(labels_treino)
  labels_teste = np.array(labels_teste)
  labels_val = np.array(labels_val)


  if model != None:
    classifier(treino, teste, val, labels_treino, labels_teste, labels_val, model)
  else:
    classifier(treino, teste, val, labels_treino, labels_teste, labels_val)



In [ ]:
def toTemporal(v_treino, v_teste, v_val, labels_treino, labels_teste, labels_val):
  v_final_treino = []
  v_final_teste = []
  v_final_val = []
  labels_final_treino = []
  labels_final_teste = []
  labels_final_val = []
  v_imgs = []

  for i, elm in enumerate(v_treino):

    r_img, g_img, b_img = cv2.split(elm)
      
    r_img = np.array(r_img)
    r_img = np.ndarray.flatten(r_img)
    
    v_imgs.append(r_img)

    if len(v_imgs) == 3:
      v_final_treino.append((v_imgs[0], v_imgs[1], v_imgs[2]))
      v_imgs.clear()


  for elm in v_teste:
    
    r_img, g_img, b_img = cv2.split(elm)
      
    r_img = np.array(r_img)
    r_img = np.ndarray.flatten(r_img)

    v_imgs.append(r_img)
  
    if len(v_imgs) == 3:
      v_final_teste.append((v_imgs[0], v_imgs[1], v_imgs[2]))
      v_imgs.clear()

  
  for elm in v_val:

    r_img, g_img, b_img = cv2.split(elm)
      
    r_img = np.array(r_img)
    r_img = np.ndarray.flatten(r_img)

    v_imgs.append(r_img)

    if len(v_imgs) == 3:
      v_final_val.append((v_imgs[0], v_imgs[1], v_imgs[2]))
      v_imgs.clear()

  for i in range(0,len(labels_treino),3):
    labels_final_treino.append(labels_treino[i])

  for i in range(0,len(labels_teste),3):
    labels_final_teste.append(labels_teste[i])

  for i in range(0,len(labels_val),3):
    labels_final_val.append(labels_val[i])

  print("Estado dos conjuntos:")
  print(f"Treino: {len(v_final_treino)} -- Labels: {len(labels_final_treino)}")
  print(f"Teste: {len(v_final_teste)} -- Labels: {len(labels_final_teste)}")
  print(f"Val: {len(v_final_val)} -- Labels: {len(labels_final_val)}")
    
  return (v_final_treino, v_final_teste, v_final_val, labels_final_treino, labels_final_teste, labels_final_val)

In [ ]:
def classifier(train_set, test_set, val_set, train_label_set, test_label_set, val_label_set, model = None, track_paths = None):

  callback_funcs = [
    tf.keras.callbacks.ModelCheckpoint(filepath = "model.{epoch:02d}-{val_accuracy:.2f}", monitor="val_accuracy", save_best_only=True),
  ]

  if(model == None):
    model = Sequential([
        TCN(nb_filters = 64, kernel_initializer="glorot_uniform", dilations = (1,2,4,8,16,32,64)),
        tf.keras.layers.Dense(units=1,activation="sigmoid")
        
    ])

    model.compile(optimizer="adam", loss = tf.losses.BinaryFocalCrossentropy(), metrics = ["accuracy"])
    print("\n\n")
    
    hist = model.fit(train_set, train_label_set, validation_data=(val_set, val_label_set), epochs=50, callbacks = callback_funcs)

    model.summary()

  yhat = model.predict(test_set)

  print("Prediction Shape:")
  evaluated = []
  expected = np.ndarray.tolist(test_label_set)

  for i, val in enumerate(yhat):
    
    if val < .5:
      evaluated.append(0)
      print(f"0    -    {test_label_set[i]}")
    else:
      evaluated.append(1)
      print(f"1    -    {test_label_set[i]}")

    image_test = cv2.imread(track_paths[i])
    cv2_imshow(image_test)
  #Salvando o modelo
  if os.path.isdir("/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage/last_model") == False:
    model.save("/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage/last_model")

  resModel(expected, evaluated)
  

def resModel(expected, evaluated):
    
  truePos=falsePos=falseNeg=trueNeg = 0
  for i, val in enumerate(evaluated):

    #Se o modelo previu a classe da imagem corretamente
    if val == expected[i]:
      #Elencando um verdadeiro positivo
      if val == 0:
        truePos += 1
      #Elencando um verdadeiro negativo
      else:
        trueNeg += 1
    #Se o modelo previu a classe da imagem incorretamente
    else:
      #Elencando um falso positivo
      if val == 0:
        falsePos += 1
      else:
        falseNeg += 1
  print("\n\n\t====== Elucidação dos Resultados ======")
  print(f"Falsos Negativos: {falseNeg}")
  print(f"Falsos Positivos: {falsePos}")
  print(f"Verdadeiros Negativos: {trueNeg}")
  print(f"Verdadeiros Positivos: {truePos}")

  print(f"\nAcurácia: {(truePos + trueNeg)/(truePos + falsePos + trueNeg + falseNeg):.2f}")
  print(f"Sensibilidade: {truePos/(truePos + falseNeg):.2f}")
  print(f"Especificidade: {trueNeg/(falsePos + trueNeg):.2f}")
  print(f"Precisão: {truePos/(truePos + falsePos):.2f}")

In [ ]:

def dataAugmentation(v_treino_doe : list, v_treino_sau):
  labels = []
  treino = []
  lenght_doe = len(v_treino_doe)
  lenght_sau = len(v_treino_sau)
  for i in range(lenght_doe):
    v_treino_doe.append(cv2.flip(v_treino_doe[i], 0))
  for i in range(lenght_doe):
    v_treino_doe.append(cv2.flip(v_treino_doe[i], 1))
  for i in range(lenght_doe):
    v_treino_doe.append(cv2.rotate(v_treino_doe[i], cv2.ROTATE_90_CLOCKWISE))
  
  for i in range(lenght_sau):
    v_treino_sau.append(cv2.flip(v_treino_sau[i], 0))
  for i in range(lenght_sau):
    v_treino_sau.append(cv2.flip(v_treino_sau[i], 1))
  for i in range(lenght_sau):
    v_treino_sau.append(cv2.rotate(v_treino_sau[i], cv2.ROTATE_90_CLOCKWISE))

  labels = [0 for x in range(len(v_treino_doe))]
  labels.extend(1 for x in range(len(v_treino_sau)))

  treino.extend(v_treino_doe)
  treino.extend(v_treino_sau)

  return (treino, labels)


In [ ]:
def numImgs(supPrint = True):
  cont_tot = 0
  cont_doentes = 0
  cont_saudaveis = 0
  cont_paciente = 0

  if supPrint == False:

    for paciente in os.listdir(path_doentes):
      path_paciente = os.path.join(path_doentes, paciente)
      paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
      for img_path in (paths_imgs):
        cont_tot += 1
        cont_doentes += 1
        cont_paciente += 1
      print(path_paciente + f": {cont_paciente}")
      cont_paciente = 0

    for paciente in os.listdir(path_saudaveis):
      path_paciente = os.path.join(path_saudaveis, paciente)
      paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
      for i, img_path in enumerate(paths_imgs):
        cont_tot += 1
        cont_saudaveis += 1
        cont_paciente += 1
      print(path_paciente + f": {cont_paciente}")
      cont_paciente = 0

    print(f"\nTotal de imagens na base: {cont_tot}")
    print(f"Total de imagens de pacientes doentes na base: {cont_doentes}")
    print(f"Total de imagens de pacientes saudáveis na base: {cont_saudaveis}")

  return (cont_tot, cont_doentes, cont_saudaveis)

In [ ]:
#Preprocessing Layer

import os, shutil
import csv
import matplotlib as mp
from PIL import Image as img
import numpy as np
from scipy.signal import wiener #modulo para preprocessamento




#Definindo caminhos de onde vou tirar os csvs brutos
abs_path = os.path.abspath(os.getcwd())#pegando o caminho absoluto(raiz)
data_path = os.path.join(abs_path, 'raw_data') # pip install opencv-python-headlesscwd/raw_data
sub = 'flir_export_2' #Escolhendo qual flir usar
sub_path = os.path.join(data_path, sub)  #cwd/raw_data/ flir_export_1(varia de acordo com o escolhido na linha anterior)  
#Definindo caminho de onde vou colocar os csvs
csv_folder = os.path.join(abs_path, 'clean_csv') #cwd/clean_csv
#Definindo caminho de onde vou colcar as imagens
img_path = os.path.join(abs_path, 'images') 





#Renomeia os csvs e os limpa

def clearCsv (sub_path):

    if os.path.exists(csv_folder): 
        shutil.rmtree(csv_folder) #Remove a pasta, caso exista
        pass

    os.mkdir(csv_folder)#cria (ou recria) o diretório para os csvs
    os.mkdir(os.path.join(csv_folder, 'doentes'))#criando subdiretório para doentes
    os.mkdir(os.path.join(csv_folder, 'nao_doentes'))#para não doentes

    for group in os.listdir(sub_path): # Doentes ou não doentes

        group_path = os.path.join(sub_path, group) # cwd/raw_data/flir_export_1/(nao_)doentes
        i = 1 # Numera os pacientes de cada grupo

        for subject in os.listdir(group_path): # Para cada paciente dentro do seu grupo

            subject_path = os.path.join(group_path, subject) # cwd/raw_data/flir_export_1/(nao_)doentes/paciente X

            # Definindo e criando caminho da pasta de destino dos CSVs do paciente atual
            if "nao" in group:
                dest_sheet_path = os.path.join(csv_folder, 'nao_doentes',
                    'nao_doente_{}'.format(i)) # cwd/clean_csv/nao_doentes/nao_doente_X
            else:
                dest_sheet_path = os.path.join(csv_folder,
                    'doentes', 'doente_{}'.format(i)) # cwd/clean_csv/doentes/doente_X
                pass

            os.mkdir(dest_sheet_path)#criando pasta de destino para os csvs dos doentes
            i += 1

            for sheet in os.listdir(subject_path):
                sheet_path = os.path.join(subject_path, sheet) # cwd/raw_data/flir_export_1/(nao_)doentes/paciente X/IR_XXXXX.csv

                relevantRows = 0
                topUselessRows = 0
                relevantCols = 0
            
             # Tratando o CSV
                with open(sheet_path) as sheet_file:
                    sheet_array = csv.reader(sheet_file, delimiter=';')
                    sheet_list = list(sheet_array)

                    # Região hepática FLIR tem 4 linhas inúteis no topo e 1 no fundo
                    # e uma coluna inútil à esquerda
                    relevantRows = len(sheet_list) - 5
                    topUselessRows = len(sheet_list) - relevantRows - 1
                    relevantCols = len(sheet_list[10]) - 1

                # Coletando o CSV sem a informação inútil
                sheet_clear = pd.read_csv(sheet_path, skiprows=topUselessRows, nrows=relevantRows, 
                usecols=[i for i in range(1,relevantCols+1)], header=None, sep=';') 
                
                # Salvando o CSV limpo no padrão ideal para converter em imagem
                sheet_clear.to_csv(os.path.join(dest_sheet_path, sheet), 
                    index=False, header=False)
            





# Controla a geração de imagens térmicas em escala de cinza a partir dos CSVs e as melhora
def generateImages():

    # Definindo o caminho de origem e destino
    source_path = csv_folder # cwd/data/csv

    # Criando os diretórios de destino
    if os.path.exists(img_path):
        shutil.rmtree(img_path)
    os.mkdir(img_path)
    os.mkdir(os.path.join(img_path, 'doentes'))
    os.mkdir(os.path.join(img_path, 'nao_doentes'))

    # Definindo a temperatura máxima e mínima dentre todos os CSVs da base
    tempMax = 0
    tempMin = 40000
    for group in os.listdir(source_path): # Doentes ou não doentes
        group_path = os.path.join(source_path, group) # cwd/source_path/(nao_)doentes
        for subject in os.listdir(group_path): # Pacientes
            subject_path = os.path.join(group_path, subject) # cwd/source_path/(nao_)doentes/(nao_)doente_X
            for sheet in os.listdir(subject_path): # Imagens
                sheet_path = os.path.join(subject_path, sheet) # cwd/source_path/(nao_)doentes/(nao_)doente_X/IR_XXXXX.csv

                sheet_array = pd.read_csv(sheet_path, header=None, decimal=",")
                sheet_array = pd.DataFrame.to_numpy(sheet_array)
                # Armazenando temperaturas sem casas decimais
                localTempMax = int(np.max(sheet_array)*1000)
                localTempMin = int(np.min(sheet_array)*1000)
                if localTempMax > tempMax:
                    tempMax = localTempMax
                if localTempMin < tempMin:
                    tempMin = localTempMin
                #numRows, numCols = sheet_array.shape ##########
                #if numRows > greaterDimension: ##########
                #    imgSquareSize  = numRows ###########
                #if numCols > greaterDimension: #######
                #    imgSquareSize = numCols ##############

    # Gerando as imagens térmicas
    for group in os.listdir(source_path): # Doentes ou não doentes
        group_path = os.path.join(source_path, group) # cwd/source_path/(nao_)doentes
        for subject in os.listdir(group_path): # Pacientes
            subject_path = os.path.join(group_path, subject) # cwd/source_path/(nao_)doentes/(nao_)doente_X
            # Definindo e criando caminho da pasta de destino das imagens do paciente atual
            if "nao" in group:
                dest_image_path = os.path.join(img_path, 'nao_doentes', subject) # cwd/data/images/nao_doentes/nao_doente_X
            else:
                dest_image_path = os.path.join(img_path, 'doentes', subject) # cwd/data/images/doentes/doente_X
            os.mkdir(dest_image_path)
            for sheet in os.listdir(subject_path): # Imagens
                sheet_path = os.path.join(subject_path, sheet) # cwd/source_path/(nao_)doentes/(nao_)doente_X/IR_XXXXX.csv

                # Coletando o CSV
                sheet_array = pd.read_csv(sheet_path, header=None, decimal=",")
                sheet_array = pd.DataFrame.to_numpy(sheet_array)

                # Convertendo o CSV para imagem térmica
                image = toThermography(sheet_array, tempMin, tempMax)

                # Melhorando a imagem
                enhanced_image = enhance(image)

                # Salvando a imagem
                output_image = img.fromarray(np.uint8(enhanced_image))
                output_image.save(os.path.join(dest_image_path, sheet.replace("csv", "jpg"))) # cwd/data/images/(nao_)doentes/(nao_)doente_X/IR_XXXXX.jpg






# Converte uma matriz de temperaturas em uma matriz de valores de cinza, mantendo proporção
def toThermography(temperatureSheet, temperatureMin, temperatureMax):

    # Definindo as escalas para conversão
    grayMax = 255
    grayMin = 0

    # Normalização linear aplicada a todas as células do array para converter
        # de valores de temperatura a valores de pixel
    normalization = lambda i: int((((int(i*1000) - temperatureMin) * (grayMax - grayMin)) / (temperatureMax - temperatureMin)) + grayMin)
    vectorized_normalization = np.vectorize(normalization)
    image = vectorized_normalization(temperatureSheet)

    return image





# Remove ruído e equaliza o histograma de uma matriz de pixels
def enhance(image):


    # Removendo ruído
    #denoised = cv2.medianBlur(image.astype(np.uint8),5)
    #denoised = cv2.GaussianBlur(image.astype(np.uint8),(5,5),0)
    denoised = wiener(image.astype(np.uint8), 5)
    rows, cols = image.shape
    # Corrigindo glitch wiener em 2 pixels de borda
    
    for i in range(rows):
        for j in range(cols):
            if i == 0 or i == 1 or i == len(image)-1 or i == len(image) - 2:
                denoised[i][j] = image[i][j]
            if j == 0 or j == 1 or j == len(image[0])-1 or j == len(image[0])-2:
                denoised[i][j] = image[i][j]
    
    
    # Equalizando histograma
    #denoised_equalizedhist = cv2.equalizeHist(denoised.astype(np.uint8))
    clahe = cv2.createCLAHE()
    denoised_equalizedhist = clahe.apply(denoised.astype(np.uint8))

    return denoised_equalizedhist




# Função de interface
def preprocess(sub_path):
    # Criando a pasta data onde ficarão as imagens e CSVs
    if not os.path.exists(img_path):
        os.mkdir(img_path)

    # Limpando os CSVs
    clearCsv(sub_path)
    # Gerando as imagens a partir dos CSVs
    generateImages()
    print("Boa execução!!!")

    
    

#chamando a função de interface
preprocess(sub_path)




Boa execução!!!


In [ ]:
#Padding Operation into the non-linear size database

def paddingImages():

  for paciente in os.listdir(path_doentes):
    path_paciente = os.path.join(path_doentes, paciente)
    print(path_paciente)
    paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
    for i, img_path in enumerate(paths_imgs):
      
      image = cv2.imread(img_path)
      
      new_height = 283
      new_width = 283
      old_height, old_width, channels = image.shape


      final_image = np.full((new_height, new_width, channels), (0,0,0), dtype = np.uint8)

      x_center = (new_width - old_width) // 2
      y_center = (new_height - old_height) // 2

      final_image[y_center:y_center+old_height, 
       x_center:x_center+old_width] = image

      new_img_name = img_path.split("/")

      os.remove(img_path)

      cv2.imwrite(path_paciente + "/" + new_img_name[-1], final_image)
  
  for paciente in os.listdir(path_saudaveis):
    path_paciente = os.path.join(path_saudaveis, paciente)
    print(path_paciente)
    paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
    for i, img_path in enumerate(paths_imgs):
      
      image = cv2.imread(img_path)
      
      new_height = 283
      new_width = 283
      old_height, old_width, channels = image.shape


      final_image = np.full((new_height, new_width, channels), (0,0,0), dtype = np.uint8)

      x_center = (new_width - old_width) // 2
      y_center = (new_height - old_height) // 2

      final_image[y_center:y_center+old_height, 
       x_center:x_center+old_width] = image

      new_img_name = img_path.split("/")

      os.remove(img_path)

      cv2.imwrite(path_paciente + "/" + new_img_name[-1], final_image)



